# Quantification under Sample Shift

In [ ]:
from pyquantification.experiments import cached_experiments, cached_test_sample_indexes

In [ ]:
configs = [
    {
        'quantifier': 'pcc',
        'rejection_limit': 'fracmax:0.5',
        'rejectors': [
            'pcc-pt',
            'pcc-apt',
            'pcc-mip',
        ],
    },
    {
        'quantifier': 'em',
        'rejection_limit': 'fracmax:0.5',
        'rejectors': [
            'em-pt',
            'em-apt',
            'em-mip',
        ],
    },
    {
        'quantifier': 'gsls',
        'rejection_limit': 'fracmax:0.5',
        'rejectors': [
            'ugsls-pt',
            'ugsls-apt',
            'ugsls-mip',
        ],
    },
]
dataset_labels = {
    'plankton': 'OPL',
    'fg-plankton': 'FPL',
    'binary-plankton': 'BPL',
}

results_dfs = []
for dataset_name in dataset_labels.keys():
    print(f'Selecting sample_idxs for {dataset_name} experiments')
    sample_idxs = cached_test_sample_indexes(
        cache_key=f'sample_rejection',
        dataset_name=dataset_name,
        max_test_n=4000,
    )
    print(f'Running {dataset_name} experiments')
    for config_num, config in enumerate(configs, start=1):
        print(f'Running experiments for {dataset_name} and config {config_num}')
        results_dfs.append(cached_experiments(
            cache_key=f'sample_{dataset_name}_{config_num}_rejection_results',
            dataset_names=[dataset_name],
            classifier_names=['logreg'],
            calibration_methods=['uncalibrated'],
            loss_weights=[0],
            gain_weights=[0],
            random_states=[0],
            shift_types=['no_shift'],
            bin_counts=['auto'],
            random_priors_options=[True],
            quantification_methods=[config['quantifier']],
            rejectors=config['rejectors'],
            rejection_limits=[config['rejection_limit']],
            classification_workers=1,
            quantification_workers=10,
            continue_on_failure=True,
            sample_idxs=sample_idxs,
        ))